In [1]:
import time
import shutil
import requests
import pandas as pd

from enum import Enum
from os import path, mkdir
from decimal import Decimal
from datetime import datetime
from pydantic import ValidationError
from pydantic.dataclasses import dataclass

In [2]:
class Timeframe(Enum):
    HOURLY = "1h"
    DAILY = "1d"
    WEEKLY = "1w"
    MONTHLY = "1M"

@dataclass
class Candle:
    time: datetime
    open: Decimal
    high: Decimal
    low: Decimal
    close: Decimal
    volume: Decimal

def exluded_symbols(market: str, symbols: list[str]) -> bool:
    """Checks if a excluded symbol is part of a given market and returns True if it is."""
    if len(symbols) > 0:
        for item in symbols:
            if market.startswith(item) or market.endswith(item):
                return True
    return False

class Binance():
    BASEURL = "https://api.binance.com"

    def __init__(self) -> None:
        pass

    def markets(self, market: str, exclude: list[str] = []) -> list[str]:
        url = Binance.BASEURL + "/api/v3/ticker/price"
        r = requests.get(url)

        if r.status_code != 200:
            raise requests.exceptions.HTTPError(r.json())

        data = filter(lambda x: x.endswith(market) and not exluded_symbols(x, exclude), map(lambda x: x.get("symbol", ""), r.json()))
        return list(data)

    def kline(self, symbol: str, interval: Timeframe) -> list[Candle]:
        url = Binance.BASEURL + "/api/v3/klines"
        payload = { "symbol": symbol, "interval": interval.value }
        r = requests.get(url, params=payload)

        if r.status_code != 200:
            raise requests.exceptions.HTTPError(r.json())

        klines = []
        for kline in r.json():
            candle = Candle(*kline[:6])
            klines.append(candle)

        return klines

In [3]:
stablecoins = ["TUSD", "BUSD", "USDC", "PAX", "USDP", "DAI", "GUSD", "USDD", "USTC", "UST", "USDS"]

binance = Binance()
markets = binance.markets("USDT", stablecoins)

print(f"Downloading {len(markets)} markets...")
market_data = {}
tf = Timeframe.DAILY
for i in range(len(markets)):
    if i % 20 == 0:
        time.sleep(5)
    market = markets[i]
    market_data[market] = binance.kline(market, Timeframe.DAILY)
print("Finished downloading.")

Finished downloading.


In [7]:
data_path = path.join("..", "data")

if path.exists(data_path):
    shutil.rmtree(data_path)

mkdir(data_path)

for market, data in market_data.items():
    df = pd.DataFrame.from_dict(data).set_index("time")
    df.to_csv(path.join(data_path, f"{market}_{tf.name.lower()}.csv"))